In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import sys
from matplotlib import pyplot as plt
from scipy import optimize, stats
import warnings

if sys.version_info[0] == 2:
    from itertools import izip
else:
    izip = zip
    xrange = range

In [2]:
def mle(x1, x2, x1err=[], x2err=[], cerr=[], s_int=True, start=(1.,1.,0.1),
        bootstrap=1000, logify=True, **kwargs):
    """
    Maximum Likelihood Estimation of best-fit parameters
    Parameters
    ----------
      x1, x2    : float arrays
                  the independent and dependent variables.
      x1err, x2err : float arrays (optional)
                  measurement uncertainties on independent and
                  dependent variables. Any of the two, or both, can be
                  supplied.
      cerr      : float array (same size as x1)
                  covariance on the measurement errors (NOT YET
                  IMPLEMENTED)
      s_int     : boolean (default True)
                  whether to include intrinsic scatter in the MLE.
      start     : tuple of floats
                  initial guess for free parameters. If `s_int` is
                  True, then po must have 3 elements; otherwise it can
                  have two (for the zero point and the slope; any other
                  elements will be discarded)
      bootstrap : int or False
                  if not False, it is the number of samples with which
                  to estimate uncertainties on the best-fit parameters
      logify    : boolean (default True)
                  whether to convert the values to log10's. This is to
                  calculate the best-fit power law. Note that the
                  result is given for the equation log(y)=a+b*log(x) --
                  i.e., the zero point must be converted to 10**a if
                  `logify=True`
      **kwargs  : dictionary (optional)
                  arguments passed to `scipy.optimize.fmin`
    Returns
    -------
      a         : float
                  Maximum Likelihood Estimate of the zero point. Note
                  that if logify=True, the power-law intercept is 10**a
      b         : float
                  Maximum Likelihood Estimate of the slope
      s         : float (optional, if s_int=True)
                  Maximum Likelihood Estimate of the intrinsic scatter
    """
    x1, x2 = np.array([x1, x2])
    n = x1.size
    if x2.size != n:
        raise ValueError('x1 and x2 must have same length')
    if len(x1err) == 0:
        x1err = 1e-8 * np.absolute(x1.min()) * np.ones(n)
    else:
        x1err = np.array(x1err)
    if len(x2err) == 0:
        x2err = 1e-8 * np.absolute(x2.min()) * np.ones(n)
    else:
        x2err = np.array(x2err)  # kz
        # x2err.array(x2err)  # theirs, bad
    if logify:
        x1, x1err = to_log(x1, x1err)
        x2, x2err = to_log(x2, x2err)

    log = np.log
    fmin = optimize.fmin

    norm = log(n * (2*np.pi)**0.5) / 2
    f = lambda x, a, b: a + b * x
    if s_int:
        w = lambda b, s, dx, dy: ((b*dx)**2 + dy**2 + s**2)**0.5
        def _loglike(p, x, y, *args):
            wi = w(p[1], p[2], *args)
            return norm + (2*log(wi) + ((y - f(x, *p[:2])) / wi)**2).sum()
    else:
        w = lambda b, dx, dy: ((b*dx)**2 + dy**2)**0.5
        def _loglike(p, x, y, *args):
            wi = w(p[1], *args)
            return norm + (2*log(wi) + ((y - f(x, *p[:2])) / wi)**2).sum()
        start = start[:2]

    fit = fmin(_loglike, start, args=(x1,x2,x1err,x2err), **kwargs)
    # bootstrap errors?
    if bootstrap is False:
        return fit
    jboot = np.random.randint(0, n, (bootstrap,n))
    boot = [fmin(_loglike, start, args=(x1[j],x2[j],x1err[j],x2err[j]),
                 disp=False, full_output=False)
            for j in jboot]
    out_err = np.std(boot, axis=0)
    out = np.transpose([fit, out_err])
    return out


def to_log(x, xerr=[], base=10, which='average'):
    """
    Take linear measurements and uncertainties and transform to log
    values.
    Parameters
    ----------
    x : array of floats
        measurements of which to take logarithms
    Optional Parameters
    -------------------
    xerr : array of floats
        uncertainties on x
    base : float
        base with which the logarithms should be calculated. FOR NOW USE
        ONLY 10.
    which : {'lower', 'upper', 'both', 'average'}
        Which uncertainty to report; note that when converting to/from
        linear and logarithmic spaces, errorbar symmetry is not
        preserved. The following are the available options:
            if which=='lower': logxerr = logx - log(x-xerr)
            if which=='upper': logxerr = log(x+xerr) - logx
        If `which=='both'` then both values are returned, and if
        `which=='average'`, then the average of the two is returned.
        Default is 'average'.
    Returns
    -------
    logx : array of floats
        values in log space, i.e., base**logx
    logxerr : array of floats
        log-uncertainties, as discussed above
    """
    if np.iterable(x):
        return_scalar = False
    else:
        return_scalar = True
        x = [x]
    x = np.array(x)
    if not np.iterable(xerr):
        xerr = [xerr]
    if len(xerr) == 0:
        xerr = np.zeros(x.shape)
    else:
        xerr = np.array(xerr)
    assert xerr.shape == x.shape, \
        'The shape of x and xerr must be the same'
    assert which in ('lower', 'upper', 'both', 'average'), \
        "Valid values for optional argument `which` are 'lower', 'upper'," \
        " 'average' or 'both'."
    logx = np.log10(x)
    logxlo = logx - np.log10(x-xerr)
    logxhi = np.log10(x+xerr) - logx
    if return_scalar:
        logx = logx[0]
        logxlo = logxlo[0]
        logxhi = logxhi[0]
    if which == 'both':
        return logx, logxlo, logxhi
    if which == 'lower':
        logxerr = logxlo
    elif which == 'upper':
        logxerr = logxhi
    else:
        logxerr = 0.5 * (logxlo+logxhi)
    return logx, logxerr

In [3]:
import pandas as pd

data = pd.read_csv(('/Users/kimzoldak/Github/GRB_Hubble_Diagram/data/'
                    'Band_13_GBM+LAT__22_GBMconstrained.txt'), 
                   sep='\t')

data = data[data.trigger.map(lambda x: x != 'bn090510016')]  # remove short burst
data.index = range(0, len(data))

df = pd.DataFrame()
df['eiso'] = data['eiso']/1E52
df['eiso_err'] = data.loc[:, ['eiso_err_low', 'eiso_err_up']].apply(np.mean, 1)
df['eiso_err'] = df['eiso_err']/1E52
df['epeakRest'] = data['epeakRest']
df['epeakRest_err'] = data.loc[:, ['epeakRest_err_low', 'epeakRest_err_up']].apply(np.mean, 1)

In [5]:
df.head()

,eiso,eiso_err,epeakRest,epeakRest_err
0,348.220960,7.055090,2731.929350,183.746653
1,410.198261,9.066845,3209.456160,243.199227
2,11.813083,0.339776,1102.755808,89.911198
3,338.085197,2.624849,2880.751218,51.922253
4,197.626135,1.059907,940.280908,12.310902


In [6]:
to_log(x=np.asarray(df['eiso']), xerr=np.asarray(df['eiso_err']), base=10, which='average')

(array([ 2.54185491,  2.61299381,  1.07236324,  2.52902616,  2.29584438,
         0.95559486,  0.377605  ,  1.95587216,  1.7817039 ,  2.22798906,
         1.77951585,  1.27529528,  1.50577426,  1.5529516 ,  1.37900051,
         0.6214454 ,  1.38880257,  0.59277242,  0.16047497,  0.80329796,
         0.60744442,  1.10608739, -0.11498411,  1.41045805,  0.93947386,
         0.39290272,  1.55206348,  0.55165741,  2.26252752,  1.28786324,
         1.09571032,  0.45143842,  1.49451712,  0.98435176]),
 array([0.00880018, 0.00960102, 0.01249492, 0.00337187, 0.00232923,
        0.01244287, 0.04578638, 0.01442056, 0.00722275, 0.00694363,
        0.00707757, 0.00386576, 0.03001827, 0.00767281, 0.02603479,
        0.01008966, 0.00669013, 0.02594262, 0.00749881, 0.03333501,
        0.07933074, 0.05161403, 0.03345206, 0.03404931, 0.02511293,
        0.03736718, 0.01840785, 0.02666182, 0.00572004, 0.04564094,
        0.02216509, 0.01215324, 0.01984748, 0.01198458]))

In [7]:
to_log(x=np.asarray(df['epeakRest']), xerr=np.asarray(df['epeakRest_err']), base=10, which='average')

(array([3.43646946, 3.50643145, 3.04247935, 3.45950575, 2.97325762,
        2.86648647, 2.33270509, 2.88375434, 2.99229292, 3.16820682,
        3.05053867, 2.55857095, 2.85059   , 2.457404  , 2.79032391,
        2.40343558, 2.43401211, 2.27210394, 1.74770618, 2.3255078 ,
        2.53830136, 2.42366128, 2.38899218, 2.43898041, 2.11096192,
        1.78440188, 2.74683131, 2.08541495, 3.46699154, 2.94254254,
        2.32751543, 2.06024854, 2.66904495, 2.30113692]),
 array([0.02925435, 0.03297223, 0.03548819, 0.00782851, 0.00568645,
        0.0262969 , 0.07435292, 0.0142109 , 0.02381436, 0.01517615,
        0.02238772, 0.00669906, 0.06643355, 0.00690299, 0.029092  ,
        0.01045163, 0.00907947, 0.01764416, 0.02456336, 0.04513165,
        0.05735654, 0.05815116, 0.01947638, 0.08597159, 0.06594926,
        0.04133206, 0.01782434, 0.07313081, 0.01196801, 0.08627908,
        0.04017442, 0.01492702, 0.02392353, 0.01693227]))

In [8]:
mle( x1 = np.asarray(df['eiso']), 
     x2 = np.asarray(df['epeakRest']), 
     x1err = np.asarray(df['eiso_err']), 
     x2err= np.asarray(df['epeakRest_err']), 
     cerr=[],
     s_int=True, 
     start=(1.,1.,1.),
     bootstrap=1000, 
     logify=True)

Optimization terminated successfully.
         Current function value: -62.342151
         Iterations: 69
         Function evaluations: 127


array([[1.93142871, 0.1017148 ],
       [0.55633529, 0.06159677],
       [0.23060952, 0.02666611]])

In [54]:
mle( x1 = np.asarray(df['eiso']), 
     x2 = np.asarray(df['epeakRest']), 
     x1err = np.asarray(df['eiso_err']), 
     x2err= np.asarray(df['epeakRest_err']), 
     cerr=[],
     s_int=True, 
     start=(1.,1.,1.),
     bootstrap=False, 
     logify=True)

Optimization terminated successfully.
         Current function value: -62.342151
         Iterations: 69
         Function evaluations: 127


array([1.93142871, 0.55633529, 0.23060952])

In [55]:
mle( x1 = np.asarray(df['eiso']), 
     x2 = np.asarray(df['epeakRest']), 
     x1err = np.asarray(df['eiso_err']), 
     x2err= np.asarray(df['epeakRest_err']), 
     cerr=[],
     s_int=False, 
     start=(1.,1.,1,),
     bootstrap=1000, 
     logify=True)

Optimization terminated successfully.
         Current function value: 5156.525643
         Iterations: 33
         Function evaluations: 64


array([[1.8054276 , 0.13768494],
       [0.57929079, 0.09089536]])

In [56]:
mle( x1 = np.asarray(df['eiso']), 
     x2 = np.asarray(df['epeakRest']), 
     x1err = np.asarray(df['eiso_err']), 
     x2err= np.asarray(df['epeakRest_err']), 
     cerr=[],
     s_int=False, 
     start=(1.,1.,1,),
     bootstrap=False, 
     logify=True)

Optimization terminated successfully.
         Current function value: 5156.525643
         Iterations: 33
         Function evaluations: 64


array([1.8054276 , 0.57929079])

## How are these results any different from what I get when I use the log-likelihood for a standard normal distribution without accounting for measurement errors?

In [57]:
def LogLikelihood(parameters, x, y):
    m,b,sigma  = parameters
    x = np.asarray(x)
    y = np.asarray(y)
    ymodel      = b + m*x
    n           = float(len(ymodel))
    err         = y-ymodel
    f = ((n*np.log(2.*np.pi*sigma**2))/2) + (np.sum((err**2)/(2.*sigma**2)))
    return f

    #f = (-0.5*n*log(2.0*pi*sigma*sigma)) - (np.sum((err**2)/(2.0 * sigma*sigma)))
    #f = (-0.5*n*log(2.*pi*sigma**2)) - (np.dot(error.T,error)/(2.*sigma**2))  
    #return -1*f



result = optimize.minimize(
                            fun=LogLikelihood, 
                            x0=np.array([1,1,1]), 
                            method='L-BFGS-B', 
                            args= (np.asarray(df['eiso'].apply(np.log10)), 
                                   np.asarray(df['epeakRest'].apply(np.log10)))
                          )

result

      fun: -0.9977060123864732
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00000000e+00,  0.00000000e+00, -1.77635684e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 96
      nit: 14
   status: 0
  success: True
        x: array([0.55595838, 1.93207634, 0.23497343])

### `array([0.55595838, 1.93207634, 0.23497343])`  (standard normal dist with no errors)
### verses 
### `array([1.93142871, 0.55633529, 0.23060952])` (accounting for errors)

### There is really no significant difference.

# So what is the point in accounting for errors if there is no difference between the function using errors and the funciton ignoring them?

```





```
## Lets re-write their MLE LogLikelihood to be set-up similar to ours:
This is just to make sure we understand their setup and how it relates to ours. 

In [67]:
# Pre-log data
xdata, xdataErr = to_log(x=np.asarray(df['eiso']), xerr=np.asarray(df['eiso_err']), which='average')
ydata, ydataErr = to_log(x=np.asarray(df['epeakRest']), xerr=np.asarray(df['epeakRest_err']), which='average')

In [68]:
def LogLikelihood_withErrors(parameters, x, y, xerr, yerr):
    p = parameters
    x = np.asarray(x)
    y = np.asarray(y)
    xerr = np.asarray(xerr)
    yerr = np.asarray(yerr)
    n = len(x)
    
    norm = np.log(n*(2*np.pi)**0.5)/2.
    ymod = lambda x, a, b: a + b * x
    sig = lambda b, s, dx, dy: ((b*dx)**2 + dy**2 + s**2)**0.5
    return norm + (2*np.log(sig(p[1], p[2], xerr, yerr)) + ((y - ymod(x, *p[:2]))/sig(p[1], p[2], xerr, yerr) )**2).sum()



result = optimize.minimize(
                            fun=LogLikelihood_withErrors, 
                            x0=np.array([1.,1.,1.]), 
                            #method='L-BFGS-B', 
                            args= (xdata, ydata, xdataErr, ydataErr)
                          )

result


      fun: -62.34215104387983
 hess_inv: array([[ 3.44707508e-03, -2.05212931e-03,  5.08421099e-06],
       [-2.05212931e-03,  1.59242249e-03, -1.14857093e-06],
       [ 5.08421099e-06, -1.14857093e-06,  4.16291563e-04]])
      jac: array([-2.86102295e-06, -2.86102295e-06,  9.53674316e-07])
  message: 'Optimization terminated successfully.'
     nfev: 105
      nit: 15
     njev: 21
   status: 0
  success: True
        x: array([1.93141678, 0.55633108, 0.23060075])

### `array([1.93142871, 0.55633529, 0.23060952])`  (Theirs)
### verses 
### `array([1.93141678, 0.55633108, 0.23060075])` (Ours)

### There is no real difference, we successfully reproduced their function. 

**If you print (wi) in their function and print (print(sig(p[1], p[2], xerr, yerr))) in mine, you get idential values. Thus, the term accounting for uncertainties is the same in our setup as theirs.** 